# IEEE LV Powerflow Solver Test

## Run simulation

In [1]:
%%bash
TOP=${TOP:-$(git rev-parse --show-toplevel)}
PATH=${TOP}/build/Examples/Cxx

IEEE_LV_PowerFlowTest ${TOP}/Examples/CIM/IEEE_EU_LV_reduced/*.xml

CIMContentHandler: Note: 0 out of 5507 tasks remain unresolved!


In [2]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import villas.dataprocessing.validationtools as validationtools
import os
import urllib.request

## Read DPsim results

In [3]:
path = 'logs/'
logName = 'IEEE_EU_LV_reduced'
dpsim_result_file = path + logName + '.csv'

ts_dpsim = read_timeseries_csv(dpsim_result_file)

# Fix for dpsim naming - TODO: unify dpsim notation in log file and update villas-dataprocessing accordingly
for ts,values in ts_dpsim.items():
    values.name = values.name[:-2]

column number: 135
results length: 5
real column names: []
complex column names: ['N0.V', 'N1.V', 'N10.V', 'N100.V', 'N101.V', 'N102.V', 'N103.V', 'N104.V', 'N105.V', 'N106.V', 'N107.V', 'N108.V', 'N109.V', 'N11.V', 'N110.V', 'N111.V', 'N112.V', 'N113.V', 'N114.V', 'N115.V', 'N116.V', 'N117.V', 'N118.V', 'N119.V', 'N12.V', 'N120.V', 'N121.V', 'N122.V', 'N123.V', 'N124.V', 'N125.V', 'N126.V', 'N127.V', 'N128.V', 'N129.V', 'N13.V', 'N130.V', 'N131.V', 'N132.V', 'N133.V', 'N134.V', 'N14.V', 'N15.V', 'N16.V', 'N17.V', 'N18.V', 'N19.V', 'N2.V', 'N20.V', 'N21.V', 'N22.V', 'N23.V', 'N24.V', 'N25.V', 'N26.V', 'N27.V', 'N28.V', 'N29.V', 'N3.V', 'N30.V', 'N31.V', 'N32.V', 'N33.V', 'N34.V', 'N35.V', 'N36.V', 'N37.V', 'N38.V', 'N39.V', 'N4.V', 'N40.V', 'N41.V', 'N42.V', 'N43.V', 'N44.V', 'N45.V', 'N46.V', 'N47.V', 'N48.V', 'N49.V', 'N5.V', 'N50.V', 'N51.V', 'N52.V', 'N53.V', 'N54.V', 'N55.V', 'N56.V', 'N57.V', 'N58.V', 'N59.V', 'N6.V', 'N60.V', 'N61.V', 'N62.V', 'N63.V', 'N64.V', 'N65.V', 'N66.V',

## Read NEPLAN results as reference

In [4]:
if not os.path.exists('reference-results'):
    os.mkdir('reference-results')

url = 'https://git.rwth-aachen.de/acs/public/simulation/reference-results/raw/master/Neplan/ReferenceGrids/IEEE_EU_LV.rlf'
neplan_result_file = 'reference-results/IEEE_EU_LV.rlf'
urllib.request.urlretrieve(url, neplan_result_file) 
print(neplan_result_file)

ts_NEPLAN = read_timeseries_NEPLAN_loadflow(neplan_result_file)
# print([ts_NEPLAN[i].name for i in range(len(ts_NEPLAN))])

reference-results/IEEE_EU_LV.rlf


## Comparison and assertion of results

In [5]:
net_name='IEEE_EU_LV_reduced'
threshold=0.5

ts_NEPLAN_standardized=validationtools.convert_neplan_to_standard_timeseries(ts_NEPLAN)
#print([[ts_NEPLAN_standardized[i].name, ts_NEPLAN_standardized[i].values] for i in range(len(ts_NEPLAN_standardized))])
ts_dpsim_standardized=validationtools.convert_dpsim_to_standard_timeseries(ts_dpsim)
#print([[ts_dpsim_standardized[i].name, ts_dpsim_standardized[i].values] for i in range(len(ts_dpsim_standardized))])
res_err=validationtools.compare_timeseries(ts_NEPLAN_standardized,ts_dpsim_standardized)
validationtools.assert_modelica_results(net_name,res_err,threshold)

N0.V
0.0
N0.VANGLE
0.0
N1.V
0.0011928234591361772
N1.VANGLE
-0.3046223374236082
N10.V
0.0010347395050941538
N10.VANGLE
-0.32085965438358777
N100.V
0.00015130172737743458
N100.VANGLE
-0.3241375021578633
N101.V
0.000462075477469603
N101.VANGLE
-0.32285955311740283
N102.V
0.000583319482554138
N102.VANGLE
-0.32715663338768247
N103.V
0.0004952065147730553
N103.VANGLE
-0.32099369551234286
N104.V
0.0006601817349827867
N104.VANGLE
-0.3228678509397145
N105.V
0.0006645465483066878
N105.VANGLE
-0.3270088298907177
N106.V
0.0007765608516771734
N106.VANGLE
-0.3268040156417357
N107.V
0.000992513967745867
N107.VANGLE
-0.3264252829301909
N108.V
0.0011082841646705521
N108.VANGLE
-0.3199358809392075
N109.V
0.001173290906397654
N109.VANGLE
-0.3261073047467281
N11.V
0.0011812622086174232
N11.VANGLE
-0.3219247190809009
N110.V
0.0011431233966775521
N110.VANGLE
-0.3179591104832679
N111.V
0.0012497629624499721
N111.VANGLE
-0.3198323221633286
N112.V
0.000499826314607831
N112.VANGLE
-0.32073310512109526
N113.V
0